<center><h1>Visual Question Answering (VQA)</h1></center>

In [0]:
#installing packages (since i'm using google colab,i'm installing from notebook itself)

#%%capture
#!pip3 install --upgrade tensorflow-gpu
#!pip install tensorflow --upgrade

In [0]:
#importing packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
import os

<h1>Mounting the Drive </h1>

In [0]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


<h1>Variables</h1>

In [0]:
currentDirectory = "/content/drive/My Drive/pcase_study_2/"
os.chdir(currentDirectory)
currentDirectory = ""
dataDirectory = currentDirectory + "data/"
imageDirectory = dataDirectory + "train2014/"
imageNumpyDirectory = dataDirectory + "VGG19_Numpy/"
modelsDirectory = currentDirectory + "Models/"
img_width = 448
img_height = 448

BATCH_SIZE = 64
BUFFER_SIZE = 300

#while True:
#    try:
#        print(len(os.listdir(imageDirectory)))
#        break
#    except:
#        print("Buffer Error")
#        continue

In [0]:
X_val = pd.read_csv(dataDirectory + 'X_val.csv')

In [0]:
def final_fun_1(X):
    """
    X Must have two elements 
    1) array of image paths 
    2) array of questions
    """
    from PythonModules.vqa import VQA, images_to_numpy, get_tokenizer_labelEncoder

    images, questions = X[0], X[1]
    #Reading the model
    model = VQA()
    #loading saved weights
    model.load_weights(modelsDirectory  + "Coattention/best_03.hdf5")
    #reading the tokenizer and encoder
    tokenizer, label_encoder = get_tokenizer_labelEncoder()

    #featurizing images using VGG Model
    image_features  = images_to_numpy(images)

    #featurizing questions
    question_features = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(questions),padding='post',maxlen=24)

    #Getting predictions from the model
    y_pred = model.predict([image_features,question_features], verbose = 1, batch_size = 64)

    #converting predictions to class labels
    class_indices = tf.math.top_k(y_pred,k=1).indices.numpy()
    predictions = []
    for idx,i in enumerate(class_indices):
        classes = np.zeros((1,1000))
        classes[0][i] = 1
        predictions.append(label_encoder.inverse_transform(classes)[0])

    #retuting predicted class label's
    return np.array(predictions)


def final_fun_2(X, y):
    """
    X Must have two elements 
    1) array of image paths 
    2) array of questions
    y => array of class labels
    """
    from PythonModules.vqa import VQA, images_to_numpy, get_tokenizer_labelEncoder

    images, questions = X[0], X[1]
    #Reading the model
    model = VQA()
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    #loading saved weights
    model.load_weights(modelsDirectory  + "Coattention/best_03.hdf5")
    #reading the tokenizer and encoder
    tokenizer, label_encoder = get_tokenizer_labelEncoder()

    #featurizing images using VGG Model
    image_features  = images_to_numpy(images)

    #featurizing questions
    question_features = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(questions),padding='post',maxlen=24)

    #encoding class labels
    y = label_encoder.transform(y)
    y = y.toarray()

    #Getting metrics from the model
    metrics = model.evaluate(x = [image_features,question_features], y = y, verbose = 1, batch_size = 64)

    #retuting metrics 
    return metrics[0],metrics[1]

In [0]:
image_paths = X_val['image_id'].apply(lambda x:  imageDirectory + 'COCO_train2014_' + '%012d.jpg' % (x)).values
questions = X_val['question'].values
answers = X_val['multiple_choice_answer']

X = (image_paths[:100], questions[:100])
y = answers[:100]

In [0]:
predicted_classes = final_fun_1(X)
print(predicted_classes)

2/2 [==============================] - 0s 9ms/step
['no' '1' 'multi' 'yes' 'bicycles' 'blue' '1' 'grazing' 'yes' 'no' 'night'
 'grass' 'hat' 'stop' 'strawberry' 'yes' 'ocean' '2' 'yes' 'kitchen' 'no'
 'yes' '3' 'yes' 'motorcycle' 'no' 'pink' 'car' 'no' 'bus stop' 'none'
 'brown' 'shirt' 'white' 'yes' 'yes' 'clay' 'sand' 'books' 'windows' 'no'
 'eating' 'yes' 'no' 'bed' 'grapes' '1' 'no' '1' 'yes' 'no' 'no' 'no' '1'
 'yes' '12:00' 'yes' 'yes' 'yes' 'white' 'no' 'surf' 'red' 'no' 'no'
 'metal' '1' '2' 'no' '3' 'cigarette' '2' 'old' 'no' 'yes' 'on desk'
 'sunny' 'parrot' 'a' 'no' 'yes' 'yes' 'yes' 'baseball' 'white' 'no' 'no'
 'black' 'yes' '1' 'left' '1' 'yes' 'no' 'drinking' 'yes' 'yes' '1' 'yes'
 'red']


In [0]:
loss, accuracy = final_fun_2(X, y)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

2/2 [==============================] - 0s 9ms/step - loss: 1.5448 - accuracy: 0.3900
Loss:  1.5448129177093506
Accuracy:  0.38999998569488525
